# Projet de collecte de données sur des sites web : Recherche de CVEs
*ALLAGLO Giovanni*\
*Etudiant Ingénieur Informatique et Cybersécurité*

Année académique : 2023-2024

lien vers le notebook : [⇲---↱](https://colab.research.google.com/drive/1KWdobNcrMR2K17kMVmd9GyArtbH2CxLs?usp=sharing)

## Contexte

La majorité des entreprises de nos jours sont confrontées au défi majeur de maintenir la résilience de leur système d'information face aux cybermenaces. Dans ce cadre, je me propose de mettre au point un mini-projet dont le but serait de collecter dans une base de données les vulnérabilités critiques (récemment exploitées ou découvertes), les exploits et les actualités du secteur.

## Objectif

L'objectif ultime de ce projet est d'établir un service interopérable avec un système d'information, capable de déclencher des alertes en lien avec les technologies du SI touchées par les actualités relatives aux tendances des cyber-menaces et aux vulnérabilités critiques.

\
## Déroulement du projet (Milestones)

### Partie 1: Collecte de données
- Etape 1 : Mise en  place d'un script de Web Scraping
- Etape 2 : Application de tests d'evasion ( afin de verifier la capacité de notre robots à éviter la detection )

### Parti 2: Persistence des données et fonctionnalités de recherche
- Etape1 : Création d'une base de données avec SQLAlquemy
- Etape2 : Mise en place d'un moteur de recherche

##Requirements

Les librairies et bibliothèques utiles seront mentionnées dans le fichier `requirements.txt`

- import requests
- import re
- import bs4
- from bs4 import BeautifulSoup
- import pandas as pd
- import yaml
- from datetime import datetime

## **Partie 1: Collecte de données**

Notre travail se base sur 3 sources :
 - [le site du CERT du gouvernement français](https://www.cert.ssi.gouv.fr/a-propos)
 - [Le site maintenu par le NIST et contenant la base de données sur les vulnérabilités repertoriées](https://nvd.nist.gov/general)
 - [Le site maintenu par Offsec et contenant des archives d'exploits-CVE](https://www.exploit-db.com/about-exploit-db)

\
Le choix de ces sources s'est fait en se basant sur leurs divers apports en données.
La première source, CERT-FR, fournit différents éléments concernant l'actualité (alertes de sécurité, menaces) ainsi que des recommandations en matière de sécurité.

La deuxième source, NVD, est un site alimenté par le National Institute of Standards and Technology et constitue un dictionnaire de CVEs. La page web sur laquelle nous travaillerons nous permettra de recueillir les 20 dernières vulnérabilités enregistrées ainsi que leur score de gravité (CVSS).

La troisième et dernière source (*facultative*) recueille les exploits de vulnérabilités et constitue un Proof of Concept(POC) sur les exploits. Ces données sont souvent utiles aux équipes d'audits.

\

Les informations relatives à ces sources (URL, description du site) seront enregistrées dans le fichier `source_URLs.yml`.

### **Mise en place des scripts pour la collecte des données : WebScrapping**

Dans cette partie du projet, voici les principales bibliothèques :
- `requests` : *Pour effectuer les requêtes HTTP*
- `pandas`   : *Pour la manipulation des données*
- `re`       : *Pour les expressions régulières*
- `bs4`      : *Pour analyser les balises HTML des sites web*
- `yaml`     : *Pour la manipulation des fichiers .yml*

\
Dans une optique fonctionnelle, nous avons créé un module pour chacune des tâches de scraping : `nvd_scrapping.py`, `certfr_scrapping.py`. Chaque module contient des fonctions (*de lecture et de normalisation*) ainsi qu'une classe qui hérite de `ScrapperInterface`.

\
La classe mère `ScrapperInterface` peut être vue comme une interface qui définit la méthode de classe `parser()` retournant les données collectées à l'issue du scraping. Elle possède également deux attributs de classe, `URL` et `SOURCE` (*initialisés à None*), qui représentent respectivement l'URL et le nom du site web ciblé. Ainsi, toutes les classes filles héritent de ces attributs et implémentent le comportement de `ScrapperInterface`.

\
Le module `nvd_scrapping.py` contient la classe `NvdScrapper` qui implémente `ScrapperInterface`. Ce module permet de faire du scraping sur le site de la National Vulnerability Database.

Le module `certfr_scrapping.py` contient la classe `CertfrScrapper` qui implémente également `ScrapperInterface`. Ce module permet de faire du scraping sur le site de CERT-FR.

#### **Démarche & Fonctionnement de la méthode `parser()`**

1. Localisation des balises HTML grâce à l'inspecteur de page intégré dans le navigateur (Firefox, Chrome, etc.).

2. Requête GET sur l'URL et récupération de la page HTML.

3. Utilisation de `BeautifulSoup` pour parser le HTML.

4. Mise en place de fonctions de lecture et de normalisation des résultats (`normalise_url`: transforme un chemin relatif d'URL en son équivalent chemin absolu, `normalise_date`: met la date au format standard `YYYY-MM-DD HH:MM:SS`, etc.).

NB : La première étape vise à repérer l'emplacement précis des données dans la page HTML, une démarche essentielle pour définir avec précision la fonction `parser()`.


En fonction des cas, la fonction `parser` prend facultativement en paramètre un *header* pour l'envoi de la requête et retourne une DataFrame (ou une liste de DataFrames) pandas avec les informations extraites du site cible.

> Pour la classe `NvdScrapper`, elle retourne des informations sur les 20 dernières CVE répertoriées par l'Institut (NIST), telles que l'identifiant, une description brève, un lien vers une ressource pour obtenir plus de détails, la date de publication, la version CVSS utilisée pour le scoring, le score CVSS, et la source (*cette information se trouve initialement dans le fichier .yml*).

> Pour la classe `CertfrScrapper`, elle retourne une liste de DataFrames, chacun contenant les informations sur une rubrique (les alertes, les recommandations, les actualités et les menaces). Chaque DataFrame contient les informations de l'identifiant de l'élément de la rubrique, de sa date de publication, du titre et de son statut (dans le cas des alertes de sécurité).


#### Tests et Visualisation des données

Deux façons d'obtenir les résultats :

1. Utiliser la méthode `parser()`.
2. Créer une instance de la classe.

NB : En instanciant une classe, nous récupérons les données ainsi que la date de collecte via les attributs `data` et `data_collect_time`.

In [14]:
import yaml
#import des classes
from scripts.nvd_scrapping import NvdScrapper
from scripts.certfr_scrapping import CertfrScrapper

#lecture des sources dans le fichier YAML
with open("files/source_URLs.yml") as sources_file:
    sources = yaml.safe_load(sources_file)

#contenu du fichier yaml
print(sources)

ModuleNotFoundError: No module named 'scrapping_interface'

##### test sur le site **nvd.nist.gov**

In [8]:
# data from NVD|NIST
url = sources["NvdScrapper"]["url"]
source = sources["NvdScrapper"]["source"]

#Mise à jour des valeurs url et source
NvdScrapper.URL = url
NvdScrapper.SOURCE = source

# Test

#Test : Instance de classe
instance_nvd = NvdScrapper()
data_nvd1 = instance_nvd.data
print(f"date de collecte {instance_nvd.data_collect_time} ")

#Test : Usage de la méthode de classe
data_nvd2 = NvdScrapper.parser()


NameError: name 'sources' is not defined

In [9]:
data_nvd2.head()

NameError: name 'data_nvd2' is not defined

In [ ]:
data_nvd1.head()

: 

##### test sur le site **cert.ssi.gouv.fr**

In [10]:
#data from CERT-FR
url = sources["CertfrScrapper"]["url"]
source = sources["CertfrScrapper"]["source"]
CertfrScrapper.URL = url
CertfrScrapper.SOURCE = source

#Test

instance_cert = CertfrScrapper()
data_cert1 = instance_cert.data
print(f"date de collecte {instance_nvd.data_collect_time} ")

data_cert2 = CertfrScrapper.parser()

NameError: name 'sources' is not defined

In [ ]:
data_cert1[0]

: 

In [ ]:
data_cert1[0]

: 

### Tests d'evasion : Autres approches & Utilisation de selenium

Nous n'avons pas eu besoin d'utiliser des techniques d'évasion dans cette première partie du projet. Cependant, gardons à l'esprit que cela devient indispensable lorsque nous cherchons à effectuer des requêtes massives (centaines de requêtes et plus) sur une URL. Certains sites utilisent des techniques telles que les CAPTCHA et reCAPTCHA lorsqu'ils suspectent des connexions provenant d'un robot (connexion excessive depuis une même IP ou un même proxy).

\
Plusieurs techniques existent pour simuler une connexion normale :

- Modifier les données d'en-tête des requêtes (modification du user-agent, par exemple).
- Utilisation de différents proxies.
- Utilisation de la bibliothèque `selenium` pour simuler une requête émanant d'un navigateur. Cette dernière peut être combinée avec les deux méthodes précédentes.
- Utilisation du Framework Scrapy (utile par sa capacité à contourner les CAPTCHAs et d'autres mesures anti-scraping)

In [ ]:
import random
# Test avec des user-agents pris aléatoirement depuis notre fichier YAML
with open("files/headers.yml") as f_headers:
    content = yaml.safe_load(f_headers)

#Constructions d'un header
browser = random.choice(["Chrome","Firefox"])
user_agents = content["user-agent"][browser]

header = content["header"][browser]
header["User-Agent"] = random.choice(user_agents)
print("en-tête utilisée: ",header)

#Test : Instance de classe
instance_nvd = NvdScrapper(header)
data_nvd1 = instance_nvd.data
print(f"date de collecte : {instance_nvd.data_collect_time} ")

: 

In [ ]:
data_nvd1.head()

: 

### Conclusion et Retour d'expérience sur la Partie 1

- Risque de blocage d'accès au site web par le programme Python lors de l'envoi répété de requêtes avec les mêmes données d'en-tête.

  > Solution : Ajuster les en-têtes | Utiliser le Framework Selenium | Utilisation du Framework Scrapy

- La langue dans laquelle les données sont renvoyées peut changer en fonction du header, ce qui peut entraîner des erreurs lorsqu'on attend un pattern regex spécifique.

  > Solution : Spécifier la préférence linguistique dans le header.

- Des erreurs peuvent survenir en fonction de l'état ou des modifications sur le site web cible.

  > Solution : Implémenter une gestion des erreurs (bloc try-exception, utilisation de conditionnelles).

Par habitude et pour assurer la cohérence logique du code, nous optons pour le typage des variables et des fonctions. Des tests fonctionnels sont également mis en place.